In [1]:
import numpy as np
from scipy import linalg

In [2]:
# MAXVOL

def MaxVol(C: np.ndarray, Iter_Limit: int = 10):
    assert C.shape[0] >= C.shape[1], "Required n >= r"

    # assume matrix has non-singular part as start
    for _ in range(Iter_Limit):
        A = C[: C.shape[1], :]
        B = C @ np.linalg.inv(A)

        ind = np.unravel_index(np.argmax(B), B.shape)
        alpha = np.round(np.abs(B[ind]), 10)

        if alpha > 1:
            C[ind[0], :], C[ind[1], :] = C[ind[1], :], C[ind[0], :]
        elif alpha == 1:
            return A
    return A


In [3]:
n = 10
r = 4

Tensor = np.fromfunction(lambda i, j: np.cos(2*i + 3*j), (n, r)).astype(float)


In [4]:
print(np.linalg.det(MaxVol(Tensor)))

6.093687054487779e-34


In [94]:
# Cross method


def Cross(A: np.ndarray, e: float):
    m, n = A.shape

    r = 0
    i, j = np.unravel_index(np.argmax(np.abs(A), axis=None), A.shape)
    U = A[i, :].reshape(1, n)
    V = (A[:, j] / A[i, j]).reshape(m, 1)
    err = A - V @ U

    I = np.arange(m)
    J = np.arange(n)
    I[i] = -1
    J[j] = -1

    i = 0
    j = 0
    while True:
        r += 1

        possible_indices = np.where((np.abs(err[i]) > np.abs(err[i, j])) & (J != -1))[0]
        if possible_indices.size:
            j = np.argmax(possible_indices)

        possible_indices = np.where(
            (np.abs(err[:, j]) > np.abs(err[i, j])) & (I != -1)
        )[0]
        if possible_indices.size:
            i = np.argmax(possible_indices)

        norm = np.linalg.norm(V @ U, ord="fro")
        
        if np.abs(err[i, j]) * ((m - r) * (n - r)) ** 0.5 <= e * norm:
            r -= 1
            return V, U

        U = np.concatenate((U, err[i, :].reshape(1, n)), axis=0)
        V = np.concatenate((V, (err[:, j] / err[i, j]).reshape(m, 1)), axis=1)

        I[i] = -1
        J[j] = -1

        err = A - V @ U


In [95]:
n = 64
m = 100

A = np.fromfunction(lambda i, j: i * 2 + j * 0.5, (m, n)).astype(float)

U, V = Cross(A, 0.01)
print(np.linalg.norm((A - U @ V), ord="fro"))


7.155233648993101e-13


In [118]:
# RSVD


def RSVD(X: np.ndarray, r: int, k: int, l: int):
    assert k >= r, "Required k >= r"
    assert l >= k, "Required l >= k"

    m, n = X.shape

    psi = np.random.normal(size=(n, k))
    phi = np.random.normal(size=(l, m))

    Z = X @ psi
    Q, R = np.linalg.qr(Z)
    W = phi @ Q
    P, T = np.linalg.qr(W)
    G = np.linalg.inv(T) @ P.T @ phi @ X

    u, s, v = np.linalg.svd(G, full_matrices=False)
    u = u[:, :r]
    s = s[:r]
    v = v[:r, :]

    return Q @ u, s, v


In [119]:
n = 64
m = 100

A = np.fromfunction(lambda i, j: i * 2 + j * 0.5, (m, n)).astype(float)

U, s, V = RSVD(A, 40, 40, 40)
print(np.linalg.norm((A - U @ np.diag(s) @ V), ord="fro"))


3.207935944368926e-11


In [106]:
def TensorFromTucker(G: np.ndarray, U: list):
    fold_part = ""
    common_part = ""
    for i in range(G.ndim):
        if i > 0:
            fold_part += ","
        fold_part += chr(97 + i) + chr(65 + i)
        common_part += chr(65 + i)

    return np.einsum(common_part + "," + fold_part, G, *U)

In [107]:
# HOSVD


def stHOSVD(Tensor: np.ndarray, e: float = 1e-2):
    U = []
    G = Tensor.copy()
    shape = list(Tensor.shape)

    for k in range(Tensor.ndim):
        Ak = np.moveaxis(G, k, 0).reshape(shape[k], -1)
        u, s, v = np.linalg.svd(Ak, full_matrices=False)

        r = s.size - np.argwhere(np.cumsum(s[::-1] ** 2) / s[0] > e)[0][0]
        shape[k] = r

        U.append(u[:, :r])
        G = np.diag(s[:r]) @ v[:r, :]
        G = G.reshape(r, *[shape[i] for i in range(len(shape)) if i != k])
    return G.T, U


In [109]:
dims = [4, 5, 3, 7]
ranks = [2, 3, 2, 1]
S = []
Tensor = np.random.normal(size=ranks)
for i in range(len(dims)):
    S.append(np.random.normal(size=(dims[i], ranks[i])))
Tensor = Tucker_to_tensor(Tensor, S)
G_hosvd, U_hosvd = stHOSVD(Tensor, e=1e-13)

T_hosvd = Tucker_to_tensor(G_hosvd, U_hosvd)
print(np.linalg.norm(T_hosvd - Tensor) / np.linalg.norm(T_hosvd))


2.3032412921928788e-15


In [111]:
# squeeze


def squeeze(G: np.ndarray, U: list, e: float = 1e-15):
    Rs = []
    for i in range(G.ndim):
        Q, R = np.linalg.qr(U[i])
        Rs.append(R)
        U[i] = Q

    G, Un = stHOSVD(TensorFromTucker(G, Rs), e)

    for i in range(G.ndim):
        Rs[i] = U[i] @ Un[i]

    return G, Rs

G, U = stHOSVD(Tensor, e=1e-13)
squeeze(G, U)


(array([[[[-122.15592113],
          [  -6.39711231]],
 
         [[ -12.59391099],
          [ -13.29638607]],
 
         [[  -6.7865466 ],
          [  10.04553534]]],
 
 
        [[[  -4.08011654],
          [  59.38416469]],
 
         [[  30.46087204],
          [ -25.2457094 ]],
 
         [[  -9.64329473],
          [ -13.54065017]]]]),
 [array([[-0.29133036,  0.79433381],
         [-0.59115889, -0.33492662],
         [ 0.51078159,  0.40074901],
         [ 0.55205068, -0.31025505]]),
  array([[-0.63971481, -0.07297752, -0.61356843],
         [ 0.7397095 ,  0.12728958, -0.60730411],
         [ 0.08690028, -0.6746461 ,  0.17156   ],
         [-0.17043436,  0.64404914, -0.01248038],
         [ 0.08363785,  0.3294313 ,  0.47447541]]),
  array([[-0.96590387, -0.24363328],
         [ 0.16389209, -0.31348076],
         [-0.20042229,  0.91780861]]),
  array([[ 0.0650679 ],
         [ 0.72422043],
         [ 0.65919911],
         [ 0.0530914 ],
         [ 0.13535143],
         [-0.091255

In [127]:
# TT-SVD https://arxiv.org/pdf/2209.02060v2


def TT_SVD(X: np.ndarray, ranks: list):
    G = X.reshape(X.shape[0], -1)

    U_r1, s_r1, V_r1 = np.linalg.svd(G, full_matrices=False)
    U_r1 = U_r1[:, : ranks[0]]
    s_r1 = s_r1[: ranks[0]]
    V_r1 = V_r1[: ranks[0], :]

    G_list = [U_r1]
    s_prev = s_r1
    V_prev = V_r1

    U_rk = None
    s_rk = None
    V_rk = None

    for k in range(1, X.ndim - 1):
        G = (np.diag(s_prev) @ V_prev).reshape(ranks[k - 1] * X.shape[k], -1)
        U_rk, s_rk, V_rk = np.linalg.svd(G, full_matrices=False)
        U_rk = U_rk[:, : ranks[k]]
        s_rk = s_rk[: ranks[k]]
        V_rk = V_rk[: ranks[k], :]

        G_list.append(U_rk.reshape(ranks[k - 1], X.shape[k], ranks[k]))

    G_list.append(np.diag(s_rk) @ V_rk)

    return G_list


N = 128
Tensor = np.fromfunction(
    lambda i, j, k: np.sin(2 * i + 3 * j + 4 * k), (N, N, N)
).astype(float)

G_1, G_2, G_3 = TT_SVD(Tensor, [10, 10, 10])

x = G_1.shape[0]
y = G_2.shape[1]
z = G_3.shape[1]

Restored = (G_1 @ ((G_2 @ G_3).reshape(G_1.shape[1], y * z))).reshape(x, y, z)

print(np.linalg.norm(Tensor - Restored))



1.548952310575486e-12


In [135]:
# TT_squeeze


def TT_squeeze(G: np.ndarray, e: float = 1e-10):
    Q, R = np.linalg.qr(G[0])
    G[0] = Q
    for i in range(1, len(G) - 1):
        G[i] = np.einsum("mk,krt->mrt", R, G[i])
        Q, R = np.linalg.qr(G[i].reshape(-1, G[i].shape[2]))
        G[i] = Q.reshape(G[i].shape[0], G[i].shape[1], -1)

    G[-1] = np.einsum("mk,kr->mr", R, G[-1])

    u, s, G[-1] = np.linalg.svd(G[-1], full_matrices=False)
    r = s.size - np.argwhere(np.cumsum(s[::-1] ** 2) / s[0] > e)[0][0]
    G[-1] = G[-1][:r, :]
    U = u[:, :r] * s[np.newaxis, :r]

    for i in range(len(G) - 2, 0, -1):
        G[i] = np.einsum("mrt,tk->mrk", G[i], U)

        n = G[i].shape[1]
        r_r = G[i].shape[2]

        G[i] = np.reshape(G[i], (-1, n * r))
        u, s, G[i] = np.linalg.svd(G[i])

        r = s.size - np.argwhere(np.cumsum(s[::-1] ** 2) / s[0] > e)[0][0]

        G[i] = G[i][:r, :]
        G[i] = np.reshape(G[i], (-1, n, r_r))
        U = u[:, :r] * s[np.newaxis, :r]

    G[0] = G[0] @ U
    return G


TT_squeeze([G_1, G_2, G_3])


[array([[ 88.01204576,  21.11002969],
        [-55.82197813,  71.24662138],
        [-41.55176654, -80.40814189],
        [ 90.40525053,  -4.32343362],
        [-33.69195149,  84.00650834],
        [-62.36365247, -65.59465177],
        [ 85.59682488, -29.41249468],
        [ -8.87804331,  90.074485  ],
        [-78.2076856 , -45.5559293 ],
        [ 73.96980522, -52.15857328],
        [ 16.64308472,  88.96717983],
        [-87.82173933, -21.88824761],
        [ 56.45039329, -70.74972983],
        [ 40.83843415,  80.77280012],
        [-90.43996365,   3.52303933],
        [ 34.43417539, -83.70500347],
        [ 61.78061734,  66.14410546],
        [-85.85379232,  28.65368298],
        [  9.67495082, -89.99238452],
        [ 77.80139196,  46.24640928],
        [-74.42855711,  51.50179067],
        [-15.85497478, -89.11102381],
        [ 87.62455231,  22.66475065],
        [-57.07438571,  70.24729524],
        [-40.12190219, -81.13113003],
        [ 90.46759105,  -2.72236903],
        [-35

In [9]:
# ALS


def ALS(Tensor: np.ndarray, T, dim_size: int, rank: int):
    for i in range(10000):
        for j in range(dim_size):
            B = np.full((1, rank), fill_value=1)
            C = np.full((rank, rank), fill_value=1)

            for k in range(dim_size):
                if k != j:
                    C = np.multiply(C, Tensor[k].T @ Tensor[k])
                    B = linalg.khatri_rao(B, Tensor[k])

            Y = B.T @ np.moveaxis(T, j, 0).reshape(T.shape[j], -1).T
            Tensor[j] = (np.linalg.pinv(C) @ Y).T

    tensor_result = np.einsum("ir,jr,kr,wr", *Tensor)
    return Tensor, np.linalg.norm(tensor_result - T) / np.linalg.norm(T)


In [10]:
N = 10

Tensor = np.fromfunction(
    lambda i, j, k, q: np.sin(2 * i + 3 * j + 4 * k) + 2 * q, (N, N, N, N)
).astype(float)

In [11]:
dim = [10, 10, 10, 10]
for j in [1, 2, 3, 4, 5, 6, 7, 8]:
    A = [np.random.normal(size=(i, j)) for i in dim]
    A, err = ALS(A, Tensor, len(dim), j)
    print(err)


0.06590261316477247
0.048732662849557254
0.016376268121962606
2.484545318702845e-14
0.016147027905277306
5.521632194789981e-12
2.9024584779153285e-05
9.57282220534171e-10
